In [1]:
import joblib

In [2]:
loaded_model = joblib.load('E:\Sharvil\Code\PBL SEM IV\Computer Vision\ml_model.pkl')
class_names = ['noGesture','ThumbsDown','ThumbsUp']

In [3]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import pyautogui
import win32gui
import win32process
import win32api
import win32con
import os
import time
import threading
from tkinter import *
import subprocess
from PIL import Image,ImageTk

In [7]:
should_exit = False

def exit_program():
    global should_exit
    start_button.config(state='normal')  # Enable the start button when exiting
    print("Inside exit_program")
    should_exit = True

thumbs_up = False
thumbs_up_2 = False
thumbs_up_3 = 0
    
thumbs_down = False
thumbs_down_2 = False
thumbs_down_3=0
tic_up = 0
tic_down = 0

def gesture_recognition():
    global should_exit
    start_button.config(state='disabled')
    
    def get_active_browser_title():
        window = win32gui.GetForegroundWindow()
        pid = win32process.GetWindowThreadProcessId(window)[1]
        handle = win32api.OpenProcess(win32con.PROCESS_QUERY_INFORMATION | win32con.PROCESS_VM_READ, False, pid)
        exe_path = win32process.GetModuleFileNameEx(handle, 0)
        return os.path.basename(exe_path)
    
    def is_doc_viewer_application(application):
        doc_viewer_application_list = ['explorer.exe', 'vlc.exe', 'AcroRd32.exe', 'FoxitReader.exe', 'NotroPDFReader.exe', 'SumatraPDF.exe', 'PDFXEdit.exe', 'WINWORD.EXE', 'EXCEL.EXE', 'POWERPNT.EXE', 'Acrobat.exe', 'notepad.exe', 'write.exe']
        return any(app.lower() in application.lower() for app in doc_viewer_application_list)
    
    
    
    
    # MediaPipe initialization
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    
    # Video capture initialization
    cam = cv.VideoCapture(0)
    global thumbs_up, thumbs_up_2, thumbs_up_3, thumbs_down, thumbs_down_2, thumbs_down_3, tic_up, tic_down
    
    def thumbsUp():
        global thumbs_up, thumbs_up_2, thumbs_up_3, tic_up
        if thumbs_up:
            if time.perf_counter() - tic_up > 1:
                thumbs_up_2 = True
                thumbs_up_3+=1
            
        else:
            thumbs_up = True
            tic_up = time.perf_counter()
    
        if thumbs_up_2:
            if thumbs_up_3 == 1:
                if active_browser_title.lower() == "vlc.exe":
                    pyautogui.press("space")
    
                else:
                    pyautogui.press("right")
                                    
                thumbs_up = False
                thumbs_up_2 = False
                thumbs_up_3 = 0
                tic_up = 0
            
                        
        # Reset thumbs-down variables when thumbs-up is detected
        thumbs_down = False
        thumbs_down_2 = False
        thumbs_down_3=0
    
    
    def thumbsDown():
        global thumbs_down, thumbs_down_2, thumbs_down_3, tic_down
        if thumbs_down:
            if time.perf_counter() - tic_down > 1:
                thumbs_down_2 = True
                thumbs_down_3+=1
            
        else:
            thumbs_down = True
            tic_down = time.perf_counter()
    
        if thumbs_down_2:
            if thumbs_down_3 == 1:    
                pyautogui.press("left")
                
                thumbs_down = False
                thumbs_down_2 = False
                thumbs_down_3=0
                tic_down = 0
                
                        
        # Reset thumbs-up variables when thumbs-down is detected
        thumbs_up = False
        thumbs_up_2 = False
        thumbs_up_3=0
    
    
    def ignore():
        global thumbs_up, thumbs_up_2, thumbs_up_3, thumbs_down, thumbs_down_2, thumbs_down_3
        
        thumbs_up=False
        thumbs_up_2 = False
        thumbs_up_3=0 
        thumbs_down = False
        thumbs_down_2 = False
        thumbs_down_3=0    
    
    # Main loop for capturing hand landmarks and making predictions
    while not should_exit:            
        success, frame = cam.read()
        if not success:
            print("No frame to show")
            continue
        
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        hands_detected = hands.process(frame_rgb)
        frame = cv.cvtColor(frame_rgb, cv.COLOR_RGB2BGR)
        
        if hands_detected.multi_hand_landmarks:
            for hand_landmarks in hands_detected.multi_hand_landmarks:
                # Extract landmarks
                landmark_row = []
                for landmark in hand_landmarks.landmark:
                    landmark_row.extend([landmark.x, landmark.y, landmark.z])
                
                # Fill empty cells if less than 21 landmarks detected
                while len(landmark_row) < 63:
                    landmark_row.extend([None, None, None])
                
                # Convert to numpy array
                hand_landmarks = np.array([landmark_row])
    
    
                active_browser_title = get_active_browser_title()
                if active_browser_title and is_doc_viewer_application(active_browser_title):
                    
                    # Predict using the trained model
                    predictions = loaded_model.predict(hand_landmarks)
                    predicted_class_index = np.argmax(predictions)
                    predicted_gesture = class_names[predicted_class_index]
                    
                    # Print predictions on the screen
                    org = (50, 50)  # Origin point for text
                    font = cv.FONT_HERSHEY_SIMPLEX
                    fontScale = 1
                    color = (255, 255, 255)  # White color
                    thickness = 2
                    lineType = cv.LINE_AA
                
                    frame = cv.putText(frame, predicted_gesture, org, font, fontScale, color, thickness, lineType)
                    print(predicted_gesture)
                    
                    # Define a dictionary mapping gesture indices to actions
                    gesture_actions = {
                        0: lambda: ignore(),  # No action for case 0
                        1: lambda: thumbsDown(),
                        2: lambda: thumbsUp()  # Assuming pattern-3 corresponds to index 2
                    }
                    
                    if predicted_class_index in gesture_actions:
                        gesture_actions[predicted_class_index]()
    
                
        
        cv.imshow("Captures", frame)
        key = cv.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to exit
            break
            
        elif key != 255:  # Check if any key is pressed
            print("Key pressed. Exiting...")
            break
        
    
    # Release video capture and close all windows
    cam.release()
    cv.destroyAllWindows()


root = Tk()
root.title('Gesture Recognition')

root.configure(borderwidth=40, relief='groove')
    

###On Screen Buttons/Frames
def thumbs_up_description():
    description_label.config(text="By showing thumbs up, user is navigated to next slide easily,")

def thumbs_down_description():
    description_l.config(text= "By showing thumbs down,user is navigated to previous slide easily,")
    
start_button = Button(root, text='Start', padx=10, pady=10, anchor=E, command= gesture_recognition)
start_button.grid(row=1, column=0)

exit_button = Button(root, text='Exit', padx=10, pady=10, anchor=W, command=exit_program)
exit_button.grid(row=1, column=2)

#frame1 = LabelFrame(root, text='camera options', padx= 10, pady=10, labelanchor='n', command=root.quit)
frame1 = LabelFrame(root, text='camera options', padx=10, pady=10, labelanchor='n')

frame1.grid(row=2, column=1)

title0 = Label(root, text='   GEST DO IT !  ', padx=20, pady=20, font=('Cooper Black', 18)).grid(row=0, column=0, columnspan=3)

#images in frame

frame0 = LabelFrame(root, text='gesture options', padx=10, pady=20, labelanchor='n')
frame0.grid(row=3, column=0, columnspan=3)

#tu_img = ImageTk.PhotoImage(Image.open(r'E:\Sharvil\Code\PBL SEM IV\Front-End\up.png'))
#add this as a parameter in tu_label ---> image = tu_img
tu_label = Label(frame0).grid(row=0, column=0)
# Create a button with the specified text and font, and associate the command
thumbs_up_button = Label(frame0, text='Document:Next Slide!'+'\n'+'PPT : Next Slide\n VLC:Pause/Play', font=('Bookman Old Style', 12))
thumbs_up_button.grid(row=0, column=2)

description_label = Label(frame0, text="", font=('Bookman Old Style', 12))
description_label.grid(row=1 , column=2)


#td_img = ImageTk.PhotoImage(Image.open('E:\Sharvil\Code\PBL SEM IV\Front-End\down.jpg'))
#add this as a parameter in td_label ---> image = td_img
td_label = Label(frame0).grid(row=2, column=2)
 #Create a button with the specified text and font, and associate the command
thumbs_down_button = Label(frame0, text='Document:Previous Page!'+'\n'+'PPT : Previous Slide', font=('Bookman Old Style', 12))
thumbs_down_button.grid(row=2, column=0)

description_l = Label(frame0, text="", font=('Bookman Old Style', 12))
description_l.grid(row=3 , column=0)

#filler0 = Label(root, row)

#Modes for radio button/creation of radio button/storing radio input
Modes = {
    ("default","0"),
    ("external","1"),
}


#Camera=StringVar()
#Camera.set("0")
#for text, mode in Modes:
    #Radiobutton(frame1, text=text, variable=Camera, value=mode).pack()


root.mainloop()


Key pressed. Exiting...
Inside exit_program
